In [1]:
import pandas as pd

In [2]:
df = pd.read_json(path_or_buf='gg2013.json')
df.head(3)

,id,text,timestamp_ms,user
0,290620657987887104,JLo's dress! #eredcarpet #GoldenGlobes,2013-01-14 00:45:38,"{'screen_name': 'Dozaaa_xo', 'id': 557374298}"
1,290620657887219713,What's making Sofia Vergara's boobs stay like ...,2013-01-14 00:45:38,"{'screen_name': 'theAmberShow', 'id': 14648726}"
2,290620657828524032,RT @FabSugar: Kerry Washington is EVERYTHING. ...,2013-01-14 00:45:38,"{'screen_name': 'SweetyPW', 'id': 35498686}"


In [3]:
data = df['text']
data[:5]

0               JLo's dress! #eredcarpet #GoldenGlobes
1    What's making Sofia Vergara's boobs stay like ...
2    RT @FabSugar: Kerry Washington is EVERYTHING. ...
3       Anne Hathaway has got me living. #GoldenGlobes
4    Jennifer Lopez's lace dress? Thoughts? #Golden...
Name: text, dtype: object

In [4]:
import re

In [5]:
retweets_freq_map = {}

def remove_retweet_prefix(line):
    # find 'RT @abc: ' where abc's length is arbitrary
    pattern = re.compile(r'\bRT @.*\b: ') 
   
    match = re.search(pattern, line)
    if match:
        # store corresponding retweet without 'RT ' prefix
        string = match.group()[3:]
        if string in retweets_freq_map:
            retweets_freq_map[string] += 1
        else:
            retweets_freq_map[string] = 1
        
    return re.sub(pattern, ' ', line)

In [6]:
def cleanse(line):
    # replace everything to ' ' except whitespace, alphanumeric character, apostrophe, hashtag, @
    return re.sub(r'[^\w\s\'#@]', ' ', line)

In [7]:
import spacy
# python3 -m spacy download en
nlp = spacy.load('en')

In [8]:
def identify_entities(text, verbose=False):
    ents = list(nlp(text).ents)
    if verbose:
        person_tags = {}
        tags = {}
        for ent in ents:    
            entity = ' '.join(t.orth_ for t in ent).strip()
            if entity not in tags:
                tags[entity]=[ent.label_]
                if ent.label_ == 'PERSON':
                    person_tags[entity]=[ent.label_]
        return tags
    else:
        return [' '.join(t.orth_ for t in ent).strip() for ent in ents]

In [9]:
def clean_entity(text):
    # remove_apostrophe
    if text.endswith("'s"):
        return text[:-2].strip()
    '''
    # remove tweet's hashtag prefix
    elif text.startswith('@'):
        return text[1:]
    # remove retweet's hashtag prefix
    elif text.startswith('RT @'):
        return text[4:]
    '''
    return text

In [10]:
def find_presenter(verbose=False):
    pattern = re.compile(r'presenter')
    entity_dict = {}
    
    num = 0
    for tweet in data:
        line = cleanse(remove_retweet_prefix(tweet))
        match = re.search(pattern, line.lower())
        if match:
            entities = identify_entities(line, False)

            if verbose:
                # print the first 5 occurrences
                if num < 5:
                    print(line.strip())
                    print(entities)
                    print()

            for entity in entities:
                entity = clean_entity(entity)
                if len(entity) > 1:
                    if entity not in entity_dict:
                        entity_dict[entity] = 1
                    else:
                        entity_dict[entity] += 1
            num += 1
            
    print('num of matches:', num)
    return entity_dict

In [11]:
%%time 
entity_dict = find_presenter(verbose=True)

What's your verdict on @ENews presenter Giuliana Rancic's Oriett Domenech dress  #goldenglobes http   t co lesP5H0F
["Giuliana Rancic 's", 'Oriett Domenech', '', '']

THISSSSSSSSSS RT @as_seenontv It's appalling that Kerry Washington is there only as a presenter and not as a nominee  #GoldenGlobes #Scandal
['Kerry Washington', '', '# GoldenGlobes #']

Jay Leno a presenter #ERedCarpet #GoldenGlobes http   t co CG30b1rp
['Jay Leno', '# ERedCarpet', 'GoldenGlobes', '', 'CG30b1rp']

Presenter  amp  Oscar Award winner Halle Berry is showstopping in a printed gown from Atelier Versace Fall 2012 at the Golden Globes right now
['Presenter', 'Oscar Award', 'Halle Berry', 'Atelier Versace', 'Fall 2012', 'the Golden Globes']

Presenter  amp  Oscar Award winner Halle Berry is showstopping in a printed gown from Atelier Versace Fall 2012 at the Golden Globes right now
['Presenter', 'Oscar Award', 'Halle Berry', 'Atelier Versace', 'Fall 2012', 'the Golden Globes']

num of matches: 323
CPU times: use

In [12]:
# top retweets (each as an entity)
sorted(retweets_freq_map.items(), key=lambda pair: pair[1], reverse=True)[:50]

[('@goldenglobes: ', 3250),
 ('@PerezHilton: ', 2012),
 ('@TheEllenShow: ', 1696),
 ('@eonline: ', 1254),
 ('@EmWatson: ', 1147),
 ('@VanityFair: ', 809),
 ('@CNNshowbiz: ', 699),
 ('@nbcsnl: ', 690),
 ('@CiudadBizarra: ', 503),
 ('@BuzzFeed: ', 491),
 ('@piersmorgan: ', 420),
 ('@nbc: ', 391),
 ('@MARLONLWAYANS: ', 384),
 ('@EW: ', 376),
 ('@buckhollywood: ', 353),
 ('@HuffingtonPost: ', 344),
 ('@vulture: ', 342),
 ('@MarilynMonroeES: ', 334),
 ('@DavidSpade: ', 304),
 ('@MTVNews: ', 294),
 ('@PimpBillClinton: ', 281),
 ('@TVGuide: ', 254),
 ('@Cosmopolitan: ', 233),
 ('@THR: ', 229),
 ('@RichardCrouse: ', 217),
 ('@prodigalsam: ', 212),
 ('@kumailn: ', 208),
 ('@DannyZuker: ', 207),
 ('@peoplemag: ', 202),
 ('@DamienFahey: ', 200),
 ('@washingtonpost: ', 197),
 ('@MHarrisPerry: ', 194),
 ('@girlsHBO: ', 193),
 ('@eonline: Best Original Song, Motion Picture: ', 192),
 ('@DougBenson: ', 191),
 ('@rogergzz: ', 185),
 ('@TVWithoutPity: ', 184),
 ('@EonlineLatino: ', 181),
 ('@HuffPostWo

In [13]:
top_100 = sorted(entity_dict.items(), key=lambda pair: pair[1], reverse=True)[:100]
top_100

[('Bill Clinton', 100),
 ('GoldenGlobes', 61),
 ('Surprise Presenter', 34),
 ('the United States', 26),
 ('42nd', 25),
 ('Amy Poehler', 18),
 ('# GoldenGlobes', 18),
 ('Will Ferrell', 17),
 ('George Clooney', 16),
 ('Kristen Wiig', 15),
 ('Hillary Clinton', 13),
 ('the Golden Globes', 8),
 ('@kerrywashington', 8),
 ('tonight', 8),
 ('Clinton', 8),
 ('Golden Globes', 8),
 ('DROP DEAD GORG', 7),
 ('# GoldenGlobes #', 5),
 ('first', 5),
 ('Ben Affleck', 5),
 ('one', 5),
 ('two', 5),
 ('Presenter', 4),
 ('Oscar Award', 4),
 ('Halle Berry', 4),
 ('Atelier Versace', 4),
 ('Fall 2012', 4),
 ('Bradley Cooper', 4),
 ('Kate Hudson', 4),
 ('Will Farrell', 4),
 ('Oscars', 4),
 ('LOOPER', 4),
 ('# goldnotsilver', 4),
 ('Amy', 4),
 ('Tina', 4),
 ('Bill', 4),
 ('Tina Fey', 3),
 ('this year', 3),
 ('Hollywood', 3),
 ('Woah', 3),
 ('Lincoln', 3),
 ('Ferrell', 3),
 ('WillFerrell', 3),
 ('EVER', 3),
 ('Will Ferrel', 3),
 ('Bill Mack', 3),
 ('Tommy Lee Jones', 3),
 ('Iron Man', 3),
 ('three', 3),
 ('Jerem

In [14]:
# pip3 install python-Levenshtein for 4-10x speedup
from fuzzywuzzy import fuzz

In [15]:
# remove golden globes from names
import pprint
names = [pair[0] for pair in top_100]

golden_globes = [name for name in names if fuzz.ratio(name.lower(), 'golden globes') > 50]
pprint.pprint(golden_globes)

['GoldenGlobes',
 '# GoldenGlobes',
 'George Clooney',
 'the Golden Globes',
 'Golden Globes',
 '# GoldenGlobes #',
 '@goldenglobes',
 'Globe',
 '# goldenglobes #']


In [16]:
for name in golden_globes:
    del entity_dict[name]
top_20 = sorted(entity_dict.items(), key=lambda pair: pair[1], reverse=True)[:20]
top_20

[('Bill Clinton', 100),
 ('Surprise Presenter', 34),
 ('the United States', 26),
 ('42nd', 25),
 ('Amy Poehler', 18),
 ('Will Ferrell', 17),
 ('Kristen Wiig', 15),
 ('Hillary Clinton', 13),
 ('@kerrywashington', 8),
 ('tonight', 8),
 ('Clinton', 8),
 ('DROP DEAD GORG', 7),
 ('first', 5),
 ('Ben Affleck', 5),
 ('one', 5),
 ('two', 5),
 ('Presenter', 4),
 ('Oscar Award', 4),
 ('Halle Berry', 4),
 ('Atelier Versace', 4)]

In [17]:
names = [pair[0] for pair in top_20]
names_clusters = []

for name in names:
    # each name starts as a cluster
    cluster = [name]
    names_to_reduce = names[:]
    names_to_reduce.remove(name)
    
    # one vs. all comparisons
    for i in names_to_reduce:
        ratio = fuzz.ratio(name.lower(), i.lower())
        # if similarity is larger than 50 or one name is contained in the other name
        if ratio > 50 or re.search(name, i, flags=re.IGNORECASE) or re.search(i, name, flags=re.IGNORECASE):
            cluster.append(i)
    
    # if multiple names are identified in one cluster
    if len(cluster) > 1:
        names_clusters.append(cluster)
    
    print(cluster)

# find names clusters that should merge
# ['Amy Poehler', 'Amy', 'Amy Poelher']
# ['Tina', 'Tina Fey']
names_clusters = ['|'.join(sorted(cluster)) for cluster in names_clusters]
names_clusters_reduced = [line.split('|') for line in list(set(names_clusters))]
names_clusters_reduced

['Bill Clinton', 'Hillary Clinton', 'Clinton']
['Surprise Presenter', 'Presenter']
['the United States']
['42nd']
['Amy Poehler']
['Will Ferrell', 'Halle Berry']
['Kristen Wiig']
['Hillary Clinton', 'Bill Clinton', 'Clinton']
['@kerrywashington']
['tonight']
['Clinton', 'Bill Clinton', 'Hillary Clinton']
['DROP DEAD GORG']
['first']
['Ben Affleck']
['one']
['two']
['Presenter', 'Surprise Presenter']
['Oscar Award']
['Halle Berry', 'Will Ferrell']
['Atelier Versace']


[['Halle Berry', 'Will Ferrell'],
 ['Presenter', 'Surprise Presenter'],
 ['Bill Clinton', 'Clinton', 'Hillary Clinton']]

In [18]:
def evaluate_by_freq_and_weight(element):
    return entity_dict[element] * len(element)

e = entity_dict.copy()
for cluster in names_clusters_reduced:
    # select the longest name
    selected_name = max(cluster, key=evaluate_by_freq_and_weight)
    cluster.remove(selected_name)
    for name in cluster:
        # if not deleted in previous cases
        if name in e and selected_name in e:
            e[selected_name] += e[name]
            del e[name]

In [19]:
top_10 = sorted(e.items(), key=lambda pair: pair[1])[-10:][::-1]
top_10

[('Bill Clinton', 121),
 ('Surprise Presenter', 38),
 ('the United States', 26),
 ('42nd', 25),
 ('Will Ferrell', 21),
 ('Amy Poehler', 18),
 ('Kristen Wiig', 15),
 ('tonight', 8),
 ('@kerrywashington', 8),
 ('DROP DEAD GORG', 7)]

In [20]:
# top 3 inferences for host
[name[0] for name in top_10][:2]

['Bill Clinton', 'Surprise Presenter']